# *Bank marketing* - маркетинговая кампания банка. 
Основной задачей  является построение модели, которая предоставляла бы возможность анализа успешности применяемой маркетинговой стратегии для разработки рекомендаций для будущих маркетинговых кампаний. Целевой переменной является *deposit* - размещение клиентом банка депозита.

Подключимся к набору данных *Bank marketing*.  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
import pylab as pl
import matplotlib
import category_encoders as ce
import warnings
warnings.filterwarnings('ignore')
import os

print(os.listdir("../input"))
sns.set(rc={'figure.figsize':(10, 8)});
df = pd.read_csv('../input/bank-marketing-dataset/bank.csv')
df.head(10)


In [ ]:
df.tail(10)

# 1. Проанализируем предложенную базу данных

Так как столбцов достаточно много транспонируем вывод для удобства изучения *Dataset*.

In [ ]:
df.head(10).T

Изучим общую информацию о датасете: тип каждого признака, наличие в данных пропусков.

In [ ]:
df.info()

Рассмотрим статистику по каждому числовому признаку.

In [ ]:
df.describe().T

Рассмотрим статистику по каждому нечисловому признаку.

In [ ]:
df.describe(include=['object'])

Извлечем целевой признак *deposit*.

In [ ]:
df.loc[0:11161, 'deposit']

Значения целевого признака - бинарные, т.е. пространство признаков может быть разделено поверхностно на две области соответсвенно двумя значениями целевого признака. Поэтому задача машинного обучения с учителем - квалификация данных.

Рассмотрим распределение абонентов по целевому признаку *deposit*.

In [ ]:
df['deposit'].value_counts(normalize=True)

In [ ]:
df['deposit'].value_counts().plot(kind='bar',label='deposit')
plt.legend()

Значения целевого признака распределены в соотношении no/yes = 0.52616/0.47384. Таким образом, по целевой переменной выборка данных является представительной, т.е. охватывающей все значения целевой переменной. 

# Проведем необходимую предобработку данных.

Создадим  бинарные признаки на основе категориальных.

In [ ]:
df = pd.get_dummies(df, columns=['month','default','housing','loan','contact'], drop_first=True)
df.head(10)

In [ ]:
df = pd.get_dummies(df, columns=['job','marital','education','poutcome','deposit'], drop_first=True)
df.head()

Проверим типы данных.

In [ ]:
df.info()

# Разобьем данные на обучающую и валидационную выборку.

In [ ]:
df_features = pd.DataFrame()

In [ ]:
y=df['deposit_yes']
X=df
y=y.to_frame()
y


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=101)
print(y.shape)
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

# Обучаем алгоритму классификации.

In [ ]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X, y)
print(model)

expected = y
predicted = model.predict(X)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

Оценим качество модели на валидационной выборке

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

Качество модели неудовлетворительное, так как значение *accuracy* невысоко.

# Настроим оптимальное число ближайших соседей в методе *kNN*

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X, y, 
                         cv=kf, scoring='accuracy')
print(scores)
mean_score = scores.mean()
print(mean_score)

Показатель качества модели равен 0.7477156757468215. Т.е. качество модели неудовлетворительное. 

Осуществим кросс-валидацию при числе соседей k ∈ [1;50].

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 50)} 
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=kf) 
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_score_

Качество модели повысилось, но несущественно. 

# Выбор метрики в методе *kNN*

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3,
random_state=17)

tree = DecisionTreeClassifier(max_depth=5, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)

tree.fit(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
knn.fit(X_train_scaled, y_train)

In [ ]:
tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

In [ ]:
knn_pred = knn.predict(X_holdout_scaled)
accuracy_score(y_holdout, knn_pred)

In [ ]:
tree_params = {'max_depth': range(1,10),
               'max_features': range(1,10)}

tree_grid = GridSearchCV(tree, tree_params,
                         cv=5, n_jobs=-1, verbose=True)

tree_grid.fit(X_train, y_train)

In [ ]:
GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
        iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(1, 10), 'max_features': range(1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [ ]:
tree_grid.best_params_

In [ ]:
tree_grid.best_score_

In [ ]:
accuracy_score(y_holdout, tree_grid.predict(X_holdout))

# Метрический метод классификации *NearestCentroid* 

In [ ]:

from sklearn.neighbors import NearestCentroid
>>> X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
>>> y = np.array([1, 1, 1, 2, 2, 2])
>>> clf = NearestCentroid()
>>> clf.fit(X, y)
NearestCentroid()
>>> print(clf.predict([[-0.8, -1]]))
[1]

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.neighbors import NearestCentroid

n_neighbors = 50


iris = datasets.load_iris()

X = iris.data[:, :2]
y = iris.target

h = .02  


cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue'])

for shrinkage in [None, .2]:
   
    clf = NearestCentroid(shrink_threshold=shrinkage)
    clf.fit(X, y)
    y_pred = clf.predict(X)
    print(shrinkage, np.mean(y == y_pred))
   
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

   
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

   
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.title("3-Class classification (shrink_threshold=%r)"
              % shrinkage)
    plt.axis('tight')

plt.show()